# Interaction Tracking Analysis

Code to analyze the output from `interactionTracker.js`: process it to determine the likely interactions and reduce the amount of data, then produce some statistics.

In [18]:
import os
import glob

def get_newest_interaction_file(directory, startsWith = "interactions_"):
    # Get list of all files in the directory
    files = glob.glob(os.path.join(directory, '*'))
    files = [file.replace("\\", "/") for file in files]
    
    # Check if the directory is empty
    if not files:
        return None

    # Filter for interaction data.
    if startsWith:
        files = [file for file in files if file.split("/")[-1].startswith(startsWith)]
    
    # Get the newest file based on modification time
    newest_file = max(files, key=os.path.getmtime)
    return newest_file

In [19]:
import json

fileDir = 'C:/Users/rafb/AppData/Roaming/Code/User/workspaceStorage/b03f3379a1447d3620f3d4a069f1d983/grandFileNavigator.grandfilenavigator'
filePath = get_newest_interaction_file(fileDir)

interactionData = []
with open(filePath) as interactionDataFile:
    for interaction in interactionDataFile.readlines():
        interactionData.append(json.loads(interaction))

print(interactionData)

[{'timeStamp': 1743188649485, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '211-245'}, {'timeStamp': 1743188649487, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '211-245', 'targetRange': '209-244'}, {'timeStamp': 1743188653372, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '209-244', 'targetRange': '208-242'}, {'timeStamp': 1743188653375, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '208-242', 'targetRange': '209-244'}, {'timeStamp': 1743188662490, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '209-244', 'targetRange': '210-244'}, {'timeStamp': 1743188662532, 'interactio

In [ ]:
import copy

# Merge scrolls together.
# A scroll is all changes of the visible ranges with less than 250 between them (i.e., covers short interruptions).
def detectScrolling(interactionData):
    maxTimeBetweenChanges = 250 # max time in ms between ChangeVisibleRanges entries to still be considered part of one scroll
    scrollingInteractionData = []

    scrollInteraction = None
    for interaction in interactionData:
        if interaction["interactionType"] != "ChangeVisibleRanges":
            if scrollInteraction is not None:
                scrollingInteractionData.append(scrollInteraction)
                scrollInteraction = None
            scrollingInteractionData.append(interaction)
        elif scrollInteraction is None or interaction["timeStamp"] - scrollInteraction["endTime"] > maxTimeBetweenChanges:
            if scrollInteraction is not None:
                scrollingInteractionData.append(scrollInteraction)
            scrollInteraction = copy.deepcopy(interaction)
        else:
            scrollInteraction["interactionType"] = "Scroll" # only triggered if multiple visible range changes close together
            scrollInteraction["endTime"] = interaction["timeStamp"]
            scrollInteraction["targetRange"] = interaction["targetRange"]

    return scrollingInteractionData

In [21]:
# TODO: First process jumps, to prevent them from being treated as scolling data.

scrollingInteractionData = detectScrolling(interactionData)
print(scrollingInteractionData)

[{'timeStamp': 1743188649485, 'interactionType': 'Scroll', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '209-244', 'endTime': 1743188649487}, {'timeStamp': 1743188653372, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '209-244', 'targetRange': '209-244', 'endTime': 1743188653375}, {'timeStamp': 1743188662490, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '209-244', 'targetRange': '295-329', 'endTime': 1743188664572}, {'timeStamp': 1743188673209, 'interactionType': 'NavigationJump', 'backwards': True, 'sourceFilePath': 'src/agent.py', 'sourceRange': '295-329', 'targetFilePath': 'src/agent.py', 'targetLine': 295}]
